<a href="https://colab.research.google.com/github/christiandatascience/Aprendizaje-por-refuerzo-simple-con-Tensorflow/blob/main/Parte_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Aprendizaje por refuerzo simple con Tensorflow**

## Parte 1: Bandido de dos brazos

## Introducción

El aprendizaje por refuerzo nos brinda la capacidad no solo de enseñarle a un agente artificial cómo actuar, sino también de permitirle aprender a través de sus propias interacciones con un entorno. Se combinan las representaciones complejas que las redes neuronales profundas pueden aprender con el aprendizaje basado en objetivos de un agente RL.

Sin embargo, aprender a construir estos agentes requiere un pequeño cambio de pensamiento para cualquiera que esté acostumbrado a trabajar en un entorno de aprendizaje supervisado. 

Se acabó la capacidad de simplemente hacer que el algoritmo emparejara ciertos estímulos con ciertas respuestas. 

Los algoritmos de RL deben permitir que el agente aprenda los emparejamientos correctos mediante el uso de **observaciones**, **recompensas** y **acciones**. Dado que ya no existe una acción correcta dada que podamos simplemente decir que es **verdadera** para que un agente tome en cualquier circunstancia, las cosas se ponen un poco complicadas. 

De ahora en adelante se trabajará en la creación y capacitación de agentes de aprendizaje por refuerzo. 

## Bandido de dos brazos

El problema de aprendizaje por refuerzo más simple es el bandido con n brazos. Esencialmente, hay n máquinas tragamonedas, cada una con una probabilidad de pago fija diferente. El objetivo es descubrir la máquina con el mejor pago y maximizar la recompensa devuelta eligiéndola siempre. Vamos a hacerlo aún más simple, al tener solo dos máquinas tragamonedas posibles para elegir. De hecho, este problema es tan simple que es más un precursor de los problemas reales de RL que uno en sí mismo. 

El problema del bandido multibrazo se puede ver como un problema de máquinas tragamonedas, como en un casino. Si tenemos un número n de máquinas tragamonedas, y cada una nos da una recompensa positiva con probabilidad p y ninguna recompensa con probabilidad (1-p), ¿podemos crear un agente que maximice las recompensas escogiendo jugar siempre en la máquinas tragamonedas que más beneficio nos vaya a proporcionar? 

La idea es la misma; tenemos un bandido con n brazos, y cada brazo tiene una probabilidad distinta de darnos una recompensa positiva. El objetivo es crear un agente que maximice esas recompensas. <a href="https://markelsanz14.medium.com/introducci%C3%B3n-al-aprendizaje-por-refuerzo-parte-1-el-problema-del-bandido-multibrazo-afe05c0c372e"> Ver más</a>

## Los aspectos típicos de una tarea que la convierten en un problema de RL son los siguientes:

1. Diferentes acciones producen diferentes recompensas. Por ejemplo, al buscar un tesoro en un laberinto, ir a la izquierda puede llevar al tesoro, mientras que ir a la derecha puede llevar a un nido de serpientes.

2. Las recompensas se retrasan con el tiempo. Esto solo significa que incluso si ir a la izquierda en el ejemplo anterior es lo correcto, es posible que no lo sepamos hasta más adelante en el laberinto.

3. La recompensa por una acción está condicionada al estado del medio ambiente. Continuando con el ejemplo del laberinto, ir a la izquierda puede ser ideal en una determinada bifurcación del camino, pero no en otras.

El bandido de n brazos es un buen punto de partida porque no tenemos que preocuparnos por los aspectos 2 y 3. En lo único en lo que tenemos que centrarnos es en aprender qué recompensas obtenemos por cada una de las acciones posibles y asegurarnos de que elegimos la óptima. En el contexto de la jerga de RL, esto se llama **aprender una política**. Vamos a usar un método llamado gradientes de políticas, en el que nuestra red neuronal simple aprende una política para elegir acciones ajustando sus pesos a través del **descenso de gradientes utilizando la retroalimentación del entorno**. 

Existe otro enfoque para el aprendizaje por refuerzo en el que los agentes aprenden funciones de valor. En esos enfoques, en lugar de aprender la acción óptima en un estado dado, el agente aprende a predecir qué tan bueno será un estado o acción dados para el agente. Ambos enfoques permiten que los agentes aprendan un buen comportamiento, pero el enfoque de gradiente de políticas es un poco más directo.



## Gradiente de política

La forma más sencilla de pensar en una red de gradiente de políticas es aquella que produce resultados explícitos. En el caso de nuestro bandido, no necesitamos condicionar estas salidas a ningún estado. Como tal, nuestra red consistirá en solo un conjunto de pesos, cada uno de los cuales corresponderá a cada uno de los posibles brazos con que jala el brazo de la máquina tragamonedas el bandido, y representará qué tan bueno piensa nuestro agente que es tirar de cada brazo. Si inicializamos estos pesos a 1, entonces nuestro agente será algo optimista sobre la recompensa potencial de cada brazo.

Para actualizar nuestra red, simplemente probaremos un brazo con una política e-greedy (consulte la Parte 7 para obtener más información sobre las estrategias de selección de acciones). Esto significa que la mayoría de las veces nuestro agente elegirá la acción que corresponda al mayor valor esperado, pero ocasionalmente, con probabilidad e, elegirá al azar. De esta forma, el agente puede probar cada uno de los diferentes brazos para seguir aprendiendo más sobre ellos. Una vez que nuestro agente ha realizado una acción, recibe una recompensa de 1 o -1.

Con esta recompensa, podemos hacer una actualización de nuestra red usando la ecuación **policy loss**:

$Pérdida = -log(π)*A$

$A$ es una ventaja y es un aspecto esencial de todos los algoritmos de aprendizaje por refuerzo. Intuitivamente, corresponde a cuánto mejor fue una acción respecto a una línea de base. En algoritmos futuros, desarrollaremos líneas de base más complejas para comparar nuestras recompensas, pero por ahora asumiremos que la línea de base es 0, y la consideraremos como la recompensa que recibimos por cada acción.

$π$ es la política. En este caso, corresponde al peso de la acción elegida.

Intuitivamente, esta función de pérdida nos permite aumentar el peso de las acciones que generaron una recompensa positiva y disminuirlo para las acciones que generaron una recompensa negativa. 

De esta forma, será más o menos probable que el agente elija esa acción en el futuro. 

Al tomar medidas, obtener recompensas y actualizar nuestra red de esta manera circular, ¡convergiremos rápidamente en un agente que puede resolver nuestro problema de bandidos! Probemos ésta afirmación.

## El bandido de múltiples brazos

Consideremos un ejemplo simple de cómo crear un agente basado en gradiente de políticas que puede resolver el problema de los bandidos con múltiples brazos.

In [ ]:
import tensorflow as tf
import numpy as np

## Los bandidos

Aquí definimos a nuestros bandidos. Para este ejemplo estamos usando un bandido de cuatro brazos. La función pullBandit genera un número aleatorio a partir de una distribución normal con una media = 0. 

Cuanto menor sea el número de bandido, más probable es que se devuelva una recompensa positiva. 

Queremos que nuestro agente aprenda a elegir siempre al bandido que le dará esa recompensa positiva.

In [ ]:
# Lista nuestros bandidos. Actualmente, el bandido 4 (índice n.º 3) 
# está configurado para proporcionar una recompensa positiva con mayor frecuencia.
bandits = [0.2,0,-0.2,-5]
num_bandits = len(bandits)
def pullBandit(bandit):
    # Obtenemos un número aleatorio.
    result = np.random.randn(1)
    if result > bandit:
        # devolvemos una recompensa positiva.
        return 1
    else:
        # devolvemos una recompensa negativa.
        return -1

## El agente

El siguiente código establece nuestro agente neuronal simple. Consiste en un conjunto de valores para cada uno de los bandidos. Cada valor es una estimación del valor de la devolución de elegir el bandido. Usamos un método de gradiente de políticas para actualizar el agente moviendo el valor de la acción seleccionada hacia la recompensa recibida.

In [ ]:
tf.compat.v1.reset_default_graph()

# Estas dos líneas establecen la parte de alimentación hacia adelante de la red. 
# Esto hace la elección real.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

# Las siguientes seis líneas establecen el procedimiento de entrenamiento. 
# Alimentamos la recompensa y la acción elegida en la red.
# para calcular la pérdida y usarla para actualizar la red.

reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

## Entrenamiento del agente

Entrenamos nuestro agente realizando acciones en nuestro entorno, y recibiendo recompensas. Usando las recompensas y las acciones, podemos saber cómo actualizar correctamente nuestra red para elegir más a menudo las acciones que generarán las recompensas más altas con el tiempo.


In [ ]:
# Establecemos el número total de episodios para entrenar a la agente.
total_episodes = 1000 

# Establecemos el marcador para bandidos a 0.
total_reward = np.zeros(num_bandits) 

# Establecemos la posibilidad de realizar una acción aleatoria.
e = 0.1 

init = tf.initialize_all_variables()

# Iniciamos el gráfico de tensorflow
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        # Elejimos una acción aleatoria o una de nuestra red.
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)

        # Conseguimos nuestra recompensa eligiendo a uno de los bandidos.
        reward = pullBandit(bandits[action]) 
        
        # Actualizamos la red.
        _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        # Actualiza nuestra cuenta corriente de puntajes.
        total_reward[action] += reward
        if i % 50 == 0:
            print ("Correr recompensa por el " + str(num_bandits) + " bandidos: " + str(total_reward))
        i+=1
print ("El agente piensa que el bandido " + str(np.argmax(ww)+1) + " es el más prometedor....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print ("...y estaba bien!")
else:
    print ("...y estaba mal!")